In [1]:
import tensorflow as tf
from tensorflow.keras import Model

import numpy as np

import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print(f'length of text: {len(text)} characters')

length of text: 1115394 characters


In [4]:
vocab = sorted(set(text))

print(f"unique characters: {len(vocab)}")

unique characters: 65


In [5]:
# create mapping from char to index and index to char
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Process the Text
text_as_int = np.array([char2idx[c] for c in text])

In [6]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)

# create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [7]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

it = iter(sequences)
seq = next(it)
seq.numpy().shape

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


(101,)

In [8]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [9]:
for input, target in dataset.take(1):
  print(f"input text: {repr(''.join(idx2char[input.numpy()]))}")
  print(f"target text: {repr(''.join(idx2char[target.numpy()]))}")

input text: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
target text: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [10]:
# Create Training Batches
# Batch Size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [11]:
# length of the vocabulary in chars
vocab_size = len(vocab)
print(f"vocab size: {vocab_size}")

# the embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

vocab size: 65


In [12]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [13]:
# define the model
class Text_Generator_Model(Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units, batch_size):
    super(Text_Generator_Model, self).__init__()
    self.embedding1 = tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None])
    self.gru1 = tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')
    self.dense1 = tf.keras.layers.Dense(vocab_size)

  def call(self, x):
    x = self.embedding1(x)
    x = self.gru1(x)
    return self.dense1(x)

In [14]:
# Create an instance of the Text_Generator_Model
text_generator = Text_Generator_Model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [15]:
# Optimizer
optimizer = tf.keras.optimizers.Adam()

In [16]:
# select metrics to measure the loss and the accuracy of the model
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [17]:
# training function
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = text_generator(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True
        )
    )
  grads = tape.gradient(loss, text_generator.trainable_variables)
  optimizer.apply_gradients(zip(grads, text_generator.trainable_variables))

  return loss

In [18]:
# Training Step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initially hidden is None
  #hidden = text_generator.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      print(f"Epoch {epoch+1} Batch {batch_n} Loss {loss}")

  # save checkpoint the model every 5 epochs
  if (epoch+1) % 5 == 0:
    text_generator.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

text_generator.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.176314830780029
Epoch 1 Batch 100 Loss 2.344959020614624
Epoch 1 Loss 2.1666
Time taken for 1 epoch 46.070024251937866 sec

Epoch 2 Batch 0 Loss 2.1505773067474365
Epoch 2 Batch 100 Loss 1.9295190572738647
Epoch 2 Loss 1.7652
Time taken for 1 epoch 44.53809642791748 sec

Epoch 3 Batch 0 Loss 1.8135883808135986
Epoch 3 Batch 100 Loss 1.6633816957473755
Epoch 3 Loss 1.6230
Time taken for 1 epoch 44.48676323890686 sec

Epoch 4 Batch 0 Loss 1.5807485580444336
Epoch 4 Batch 100 Loss 1.5450671911239624
Epoch 4 Loss 1.4821
Time taken for 1 epoch 44.34568119049072 sec

Epoch 5 Batch 0 Loss 1.467478632926941
Epoch 5 Batch 100 Loss 1.4700640439987183
Epoch 5 Loss 1.4317
Time taken for 1 epoch 44.323925733566284 sec

Epoch 6 Batch 0 Loss 1.3793129920959473
Epoch 6 Batch 100 Loss 1.3912146091461182
Epoch 6 Loss 1.3978
Time taken for 1 epoch 44.563655853271484 sec

Epoch 7 Batch 0 Loss 1.3404607772827148
Epoch 7 Batch 100 Loss 1.3995932340621948
Epoch 7 Loss 1.3812
Time taken

In [23]:
# Restore the Latest Checkpoint
model = Text_Generator_Model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [24]:
# function to test the model to generate text
def generate_text(model, start_string):
  # Number of characters to generate
  num_generate = 1000

  #converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text
  # Higher temperatures results in more surprising text
  # Ezperiment to find the best setting
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    # remove batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions /= temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])
  
  return (start_string + ''.join(text_generated))

In [26]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: Proceed!
What laws to be charge? now 'tis a puar, thy hands
Duef. But make poesers' man do now shong to beg.

KING HENRY VI:
I am served to myself in me; come, has put
Theary you had.

EXTON:
Up as shall, save it, till the proud malice
I have in peach and qual of him?

QUEEN:
Om calling comes in the battars be thy man,
Which'd by Having under firm-more celecomat of the bean
Upon my hence, do no harm both, your gross
put thou of herfillingly.

CAMILLO:
Be not instrument to the buhied welk
Heir to have my death in him.

MERCULIO:
I'll strike at us not soonour own to be
God keep to have a kingdom sir,
For uncred her world, my protestrong of Venot I'll dissemble him;
And, our sistence comes to be ours.

GREMIO:
Yes, my lord.

ARIEL:
I'll seem to painted him where:
I have alive.

DUKE VINCENTIO:
You wood under and putt to did but die: say 'twere
He was not to brighbours to the seproos; for then cloudieve
'O cell, good sword, in dear daughter, then she gallant us to wear you have leis